<a href="https://colab.research.google.com/github/kinfkong/katago-colab/blob/master/colab_katago_perfect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 请先点击顶部的“复制到驱动器” 另存为自己的副本。


下面的代码是在colab上配置katago、ssh等环境等。  

在运行前，要准备好三个变量的值: NGROK_TOKEN及USER_NAME,USER_PASSWORD  

* NGROK_TOKEN - 在ngrok里注册登陆后，在这个页面可以看到你的token. https://dashboard.ngrok.com/auth/your-authtoken
* USER_NAME及USER_PASSWORD - 你自己给随便自己起一个用户名和密码（即USER_NAME，USER_PASSWORD) 不要有空格之类的特殊字符(最好只有英文及数字）。这个USER_NAME, USER_PASSWORD你以后配置sabaki, Lizzie之类需要用到.

弄出上面这些值后，把下面代码中的前三行分别替换成你的值。（注意引号要保留）。然后运行即可。

In [ ]:
NGROK_TOKEN="254Hvo4iQ6MWLCCw7egfv_6MXbj4h6mbD6w2Cx3ymkk"
USER_NAME="testuser"
USER_PASSWORD="87654321"


# optional args
# supports: OPENCL, CUDA or AUTO
KATAGO_BACKEND="AUTO"
# supports: 40b, 30b, 20b, 40b-large or AUTO
WEIGHT_FILE="AUTO" 

import subprocess
if KATAGO_BACKEND == "AUTO":
  gpu_name=str(subprocess.check_output("nvidia-smi -q | grep \"Product Name\" | cut -d\":\" -f2 | tr -cd '[:alnum:]._-'", shell=True), encoding='utf-8')
  if gpu_name == "TeslaT4":
    KATAGO_BACKEND="CUDA"
  else:
    KATAGO_BACKEND="OPENCL"
!echo "Using Katago Backend: " $KATAGO_BACKEND

if WEIGHT_FILE == "AUTO":
  gpu_name=str(subprocess.check_output("nvidia-smi -q | grep \"Product Name\" | cut -d\":\" -f2 | tr -cd '[:alnum:]._-'", shell=True), encoding='utf-8')
  if gpu_name == "TeslaK80" or gpu_name == "TeslaP4":
    WEIGHT_FILE="20b"
  else:
    WEIGHT_FILE="40b"

!echo "Using Katago Weight: " $WEIGHT_FILE

weight_urls = {
    '40b': 'https://github.com/lightvector/KataGo/releases/download/v1.4.5/g170-b40c256x2-s5095420928-d1229425124.bin.gz',
    '30b': 'https://github.com/lightvector/KataGo/releases/download/v1.4.5/g170-b30c320x2-s4824661760-d1229536699.bin.gz',
    '20b': 'https://github.com/lightvector/KataGo/releases/download/v1.4.5/g170e-b20c256x2-s5303129600-d1228401921.bin.gz',
    '40b-large': 'https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/40b384.bin.gz'
}

# Install useful stuff
! apt install --yes ssh screen nano htop ranger git libzip4 1>/dev/null
! pip install oss2 1>/dev/null
%cd /content

!rm -rf katago-colab
!git clone https://github.com/kinfkong/katago-colab.git 1>/dev/null

#download the binarires
!wget --quiet https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/katago-$KATAGO_BACKEND -O katago
!chmod +x /content/katago
!wget --quiet https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/ngrok -O ngrok
!chmod +x /content/ngrok
!mkdir -p /root/.katago/
!cp -r /content/katago-colab/opencltuning /root/.katago/

#download the weights
weight_url = weight_urls[WEIGHT_FILE]
!wget --quiet $weight_url -O $WEIGHT_FILE".bin.gz" 
!rm -rf weight.bin.gz
!ln -s $WEIGHT_FILE".bin.gz" weight.bin.gz

# SSH setting
! echo "root:$USER_PASSWORD" | chpasswd
! echo "PasswordAuthentication yes" > /etc/ssh/sshd_config
! echo "PermitUserEnvironment yes" >> /etc/ssh/sshd_config
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config

! mkdir -p /root/.ssh

# generate the keys
#!ssh-keygen -q -t rsa -N '' -f /root/.ssh/id_rsa <<<y 2>&1 >/dev/null
#!cat /root/.ssh/id_rsa.pub >> /root/.ssh/authorized_keys

! service ssh restart > /dev/null

# Run ngrok
get_ipython().system_raw('./ngrok authtoken $NGROK_TOKEN && ./ngrok tcp 22 &')
! sleep 5

import oss2
import requests
import json
from re import sub
r = requests.get('http://localhost:4040/api/tunnels')
raw_ssh = r.json()['tunnels'][0]['public_url']
ssh_args = (sub("tcp://", "", raw_ssh)).split(':')
ssh_option = {
    'host': ssh_args[0],
    'port': int(ssh_args[1]),
    'user': 'root'
}

ssh_option_json = json.dumps(ssh_option)

endpoint = 'http://oss-cn-beijing.aliyuncs.com'
auth = oss2.Auth('LTAI4G3tthX2R3Z8KY6Bdbs3', 'Q22UstMdKX8zZY9BqkGWWd2XbEBvPH')
bucket = oss2.Bucket(auth, endpoint, 'kata-config')

key = USER_NAME + '.ssh.json'
bucket.put_object(key, ssh_option_json)

!echo "done!"



#用Sabaki或Lizzie连接  

## 第一步: 下载应用程序
下载地址如下:  
**Windows用户 (64bit windows)**  
https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/colab-katago.windows.zip  
**Linux用户**  
https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/colab-katago.linux.zip  
**Mac OSX用户**  
https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/colab-katago.mac.zip  


**下载后，把压缩包解压，可以得到一个可执行文件: colab-katago或叫colab-katago.exe**

## 第二步: 配置Sabaki及Lizzie
配置需要设置刚才你下载来的程序的全路径，及你之前步骤里你自己设的用户名、密码(即USER_NAME, USER_PASSWORD) 

Sabaki样例:  
![替代文字](
https://kata-config.oss-cn-beijing.aliyuncs.com/images/Snip20200724_4.png)  
Lizzie样例:  
![替代文字](https://kata-config.oss-cn-beijing.aliyuncs.com/images/Snip20200724_8.png%202020-07-24%2013-24-08.png)

## 更多配置 （可以不看） 
可以用以下的参数，来更改对弈时限制每手棋的visits或时间。比如  
```
<程序全路径> <用户名> <密码> 30s
```
表示限制每步用时30秒
```
<程序全路径> <用户名> <密码> 1600v
```
表示限制每步用1600 visits的数量

上面例子中的30, 1600可以自己换成任意数字

#以下的都是调试用的代码，普通用户可以忽略下面的所有内容

显示GPU类型

In [ ]:
!nvidia-smi

获取ssh登陆信息  
*ssh登陆帐号用root, 密码是你之前设置的USER_PASSWORD*

In [ ]:
import requests
from re import sub
r = requests.get('http://localhost:4040/api/tunnels')
str_ssh = r.json()['tunnels'][0]['public_url']
str_ssh = sub("tcp://", "", str_ssh)
str_ssh = sub(":", " -p ", str_ssh)
str_ssh = "ssh root@" + str_ssh 
print(str_ssh)

ssh root@0.tcp.ngrok.io -p 12828
